In [ ]:
# Import the dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
# Read the CSV into a DataFrame
df=pd.read_csv("/content/sales.csv")
df.head()

,PageVisitsCat1,PageVisitDurationCat1,PageVisitsCat2,PageVisitDurationCat2,PageVisitsCat3,PageVisitDurationCat3,BounceRates,ExitRates,PageValues,HowCloseToSpecialDay,Month,VisitorType,Weekend,Revenue
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.0,0.0,Feb,Returning_Visitor,False,False
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.0,0.0,Feb,Returning_Visitor,False,False
2,0.0,0.0,0.0,0.0,2.0,2.666667,0.050000,0.140000,0.0,0.0,Feb,Returning_Visitor,False,False
3,0.0,0.0,0.0,0.0,10.0,627.500000,0.020000,0.050000,0.0,0.0,Feb,Returning_Visitor,True,False
4,0.0,0.0,0.0,0.0,19.0,154.216667,0.015789,0.024561,0.0,0.0,Feb,Returning_Visitor,False,False


In [ ]:
# List the columns
for col in df:
  print(col)

PageVisitsCat1
PageVisitDurationCat1
PageVisitsCat2
PageVisitDurationCat2
PageVisitsCat3
PageVisitDurationCat3
BounceRates
ExitRates
PageValues
HowCloseToSpecialDay
Month
VisitorType
Weekend
Revenue


In [1]:

# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark-Homework").getOrCreate()

In [39]:
from pyspark import SparkFiles
df = spark.read.csv("/content/sales.csv", header=True)
df.show(200)

+--------------+---------------------+--------------+---------------------+--------------+---------------------+--------------------+--------------------+------------------+--------------------+-----+-----------------+-------+-------+
|PageVisitsCat1|PageVisitDurationCat1|PageVisitsCat2|PageVisitDurationCat2|PageVisitsCat3|PageVisitDurationCat3|         BounceRates|           ExitRates|        PageValues|HowCloseToSpecialDay|Month|      VisitorType|Weekend|Revenue|
+--------------+---------------------+--------------+---------------------+--------------+---------------------+--------------------+--------------------+------------------+--------------------+-----+-----------------+-------+-------+
|           0.0|                  0.0|           0.0|                  0.0|           1.0|                  0.0|                 0.2|                 0.2|               0.0|                 0.0|  Feb|Returning_Visitor|  False|  False|
|           0.0|                  0.0|           0.0|       

In [20]:
df.createOrReplaceTempView("customer")

### Description of columns:

**PageVisitsCatX:** Number of pages visited by a visitor. Cat1 is administrative, Cat2 is informational, and Cat3 is product related.

**PageVisitDurationCatX:** How long a visitor stayed on the category of pages.

**BounceRates:** Percentage of visitors who landed and exited a page.

**ExitRates:** Percentage of visitors who left the site from that page.

**PageValues:** A measurement of a page's contribution to a sale.

**HowCloseToSpecialDay:** How close is browsing date to a special day or a holiday, such as Valentine's day? Higher numbers are closer.

**Month:** Month of visit.

**VisitorType:** Is the visitor a new or returning visitor?

**Weekend:** Did the visit occur on a weekend day?

**Revenue:** Did the visit conclude in a sale?

## Perform Data Analysis

In [38]:

# Retrieve the number visits for each month.

df.groupBy("Month").count().show()

+-----+-----+
|Month|count|
+-----+-----+
|  Oct|  549|
|  Sep|  448|
|  Dec| 1727|
|  Aug|  433|
|  May| 3357|
| June|  288|
|  Feb|  171|
|  Nov| 2995|
|  Mar| 1884|
|  Jul|  431|
+-----+-----+



In [43]:
# Retrieve the number of visits that resulted in a purchase or not.
# df.groupBy("Revenue").count().show()

sql="""
SELECT Count(Revenue) as PurchaseVisit
fROM customer
Where Revenue=True
"""
spark.sql(sql).show()

+-------------+
|PurchaseVisit|
+-------------+
|         1908|
+-------------+



In [67]:
# What percentage of visits resulted in a purchase?
sql="""
SELECT Count(Revenue) as PurchaseVisit, Revenue
fROM customer
GROUP BY Revenue
"""
spark.sql(sql).show()

#15% of visit resulted in a purchase

+-------------+-------+
|PurchaseVisit|Revenue|
+-------------+-------+
|        10375|  False|
|         1908|   True|
+-------------+-------+



In [55]:
# Filter the DataFrame for all purchases.
sql="""
SELECT *
FROM customer
Where Revenue=True
"""
spark.sql(sql).show()

+--------------+---------------------+--------------+---------------------+--------------+---------------------+-----------+--------------------+------------------+--------------------+-----+-----------------+-------+-------+
|PageVisitsCat1|PageVisitDurationCat1|PageVisitsCat2|PageVisitDurationCat2|PageVisitsCat3|PageVisitDurationCat3|BounceRates|           ExitRates|        PageValues|HowCloseToSpecialDay|Month|      VisitorType|Weekend|Revenue|
+--------------+---------------------+--------------+---------------------+--------------+---------------------+-----------+--------------------+------------------+--------------------+-----+-----------------+-------+-------+
|           3.0|          87.83333333|           0.0|                  0.0|          27.0|    798.3333332999999|        0.0|         0.012643678|        22.9160357|                 0.8|  Feb|Returning_Visitor|  False|   True|
|          10.0|          1005.666667|           0.0|                  0.0|          36.0|      

In [60]:
# What is the number of visits for each month that resulted in a purchase?
# Which month had the greatest number of visits where a purchase was made?
sql="""
SELECT Month, count(Revenue) as Purchase
FROM customer
GROUP BY Month;
"""
spark.sql(sql).show()

# Nov has the highest purchase

+-----+--------+
|Month|Purchase|
+-----+--------+
|  Oct|     115|
|  Sep|      86|
|  Dec|     216|
|  Aug|      76|
|  May|     365|
| June|      29|
|  Feb|       3|
|  Nov|     760|
|  Mar|     192|
|  Jul|      66|
+-----+--------+



In [65]:
# How many visits were from returning visitors?
# How many visits were from new visitors?
sql="""
SELECT VisitorType as Visit, count(VisitorType) as Count
FROM customer
GROUP BY Visit;
"""
spark.sql(sql).show()

+-----------------+-----+
|            Visit|Count|
+-----------------+-----+
|      New_Visitor| 1694|
|            Other|   85|
|Returning_Visitor|10504|
+-----------------+-----+



In [70]:
# How many visits took place on a weekday? And, on the weekend?
#Visit Took Place in Weekdays

sql="""
SELECT count(Weekend) as weekdayVisit
FROM customer
Where Weekend=False
"""
spark.sql(sql).show()

sql="""
SELECT count(Weekend) as weekendVisit
FROM customer
Where Weekend=True
"""
spark.sql(sql).show()

+------------+
|weekdayVisit|
+------------+
|        9421|
+------------+

+------------+
|weekendVisit|
+------------+
|        2862|
+------------+



## Perform Logistic Regression 

In [ ]:
# Convert categorical variables to binary variables 
df_one =pd.get_dummies(df)
df_one

,PageVisitsCat1,PageVisitDurationCat1,PageVisitsCat2,PageVisitDurationCat2,PageVisitsCat3,PageVisitDurationCat3,BounceRates,ExitRates,PageValues,HowCloseToSpecialDay,...,Month_Jul,Month_June,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep,VisitorType_New_Visitor,VisitorType_Other,VisitorType_Returning_Visitor
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
2,0.0,0.0,0.0,0.0,2.0,2.666667,0.050000,0.140000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
3,0.0,0.0,0.0,0.0,10.0,627.500000,0.020000,0.050000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
4,0.0,0.0,0.0,0.0,19.0,154.216667,0.015789,0.024561,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12278,3.0,145.0,0.0,0.0,53.0,1783.791667,0.007143,0.029031,12.241717,0.0,...,0,0,0,0,0,0,0,0,0,1
12279,0.0,0.0,0.0,0.0,5.0,465.750000,0.000000,0.021333,0.000000,0.0,...,0,0,0,0,1,0,0,0,0,1
12280,0.0,0.0,0.0,0.0,6.0,184.250000,0.083333,0.086667,0.000000,0.0,...,0,0,0,0,1,0,0,0,0,1
12281,4.0,75.0,0.0,0.0,15.0,346.000000,0.000000,0.021053,0.000000,0.0,...,0,0,0,0,1,0,0,0,0,1


In [ ]:
# List the columns
for col in df_one:
  print(col)

PageVisitsCat1
PageVisitDurationCat1
PageVisitsCat2
PageVisitDurationCat2
PageVisitsCat3
PageVisitDurationCat3
BounceRates
ExitRates
PageValues
HowCloseToSpecialDay
Weekend
Revenue
Month_Aug
Month_Dec
Month_Feb
Month_Jul
Month_June
Month_Mar
Month_May
Month_Nov
Month_Oct
Month_Sep
VisitorType_New_Visitor
VisitorType_Other
VisitorType_Returning_Visitor


In [ ]:
# Separate the features and target variables.
y=df_one["Revenue"]
X=df_one.drop(columns="Revenue")

In [ ]:
# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(9212, 24)

In [ ]:
# Instantiate a logistic regression model
# Create a logistic regression classifier. 
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

LogisticRegression(random_state=1)

In [ ]:
# Train the model
classifier.fit(X_train, y_train)
LogisticRegression(random_state=1)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=1)

In [ ]:
# Use the testing data to make predictions.
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions,
                        "Actual": y_test}).reset_index(drop=True)
results.head(200) 

,Prediction,Actual
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
195,True,True
196,False,False
197,False,False
198,False,False


In [ ]:
# Calculate the accuracy. 
# Get the training and testing data score.
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.882544507164568
Testing Data Score: 0.8922175187235428


In [ ]:
# Print the confusion matrix.

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[2536,   58],
       [ 273,  204]])

In [ ]:
# Print a classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

       False       0.90      0.98      0.94      2594
        True       0.78      0.43      0.55       477

    accuracy                           0.89      3071
   macro avg       0.84      0.70      0.75      3071
weighted avg       0.88      0.89      0.88      3071



## Overall Model Performance
----

- **Question:** Can the logistic regression model predict whether visiting a website will result in a purchase? 

- **Answer:**   Yes, Logistic regression has the highest accuracy score
